# LSTM을 통한 노래 작문
- 노래 가사 데이터들을 로드한다.
- 노래 가사 데이터의 전처리를 한다.
  - 가사의 단어가 15개를 넘는 것은 제외한다.
  - 한줄의 단어가 0인것은 pass한다.
  - 특수문자들은 공백으로 변환한다.
  - 대문자는 모두 소문자로 변환한다.
- 데이터를 train set과 val set으로 분리한다.
  - 분리한 데이터들은 tensorflow dataset 형태로 변환
- batch size 256 hidden 2048 LSTM 두개의 layer을 사용
  - 오버피팅의 방지를 위해 dropout(0.3)을 사용
- evaluation을 통해 val set을 평가한다.

In [38]:
import os, re 
import numpy as np
import tensorflow as tf
import glob
# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
# file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
# with open(file_path, "r") as f:
#     raw_corpus = f.read().splitlines()

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])



데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you']


## 데이터 전처리
- 가사의 단어가 15개를 넘는 것은 제외한다.
  - 한줄의 단어가 0인것은 pass한다.
  - 특수문자들은 공백으로 변환한다.
  - 대문자는 모두 소문자로 변환한다.

In [2]:
def preprocess_sentence(raw_corpus):
    sentence = raw_corpus.lower().strip() # 1        
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4 알파벳이 아닌 모든 문자
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6    
    return sentence

# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
#     if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split(" ")) > 15 : continue
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]
print(corpus[0].split(" "))

['<start>', 'at', 'first', 'i', 'was', 'afraid', '<end>']


## 데이터 토큰화
- 문장의 시작에 \<start\> 를 넣고 끝에 \<end\>를 넣는다.

In [3]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   70  247 ...    0    0    0]
 [   2    4   53 ...    0    0    0]
 [   2    4 1066 ...    0    0    0]
 ...
 [   2    8    4 ...    0    0    0]
 [   2   44   17 ...    0    0    0]
 [   2    6  174 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fba7eb2f590>


In [4]:
print(tensor[:3, :10])

[[   2   70  247    4   53  708    3    0    0    0]
 [   2    4   53 6269    3    0    0    0    0    0]
 [   2    4 1066  525    4  104   80  192  257    7]]


In [5]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  70 247   4  53 708   3   0   0   0   0   0   0   0]
[ 70 247   4  53 708   3   0   0   0   0   0   0   0   0]


## 데이터 tf.data.Dataset 형태로 변환

In [7]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
print(steps_per_epoch)
 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

610


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 모델 설계
- LSTM 2개 embedding size 256 hidden size 2048 사용

In [31]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.dropout_1 = tf.keras.layers.Dropout(0.2)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.dropout_2 = tf.keras.layers.Dropout(0.2)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.dropout_1(out)
        out = self.rnn_2(out)
        out = self.dropout_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

## 데이터셋 분리
- 0.2로 train set과 val set을 분리한다.
- 각각 분리된 set은 datset 모듈로 변환한다.

In [32]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,test_size=0.2, random_state=5)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Val:", enc_val.shape)
print("Target Val:", dec_val.shape)

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
# dataset

dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset_val.shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)


for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

Source Train: (124981, 14)
Target Train: (124981, 14)
Source Val: (31246, 14)
Target Val: (31246, 14)


<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.2915002e-04, -8.4298663e-05, -2.0246314e-04, ...,
         -4.0677792e-04,  1.0153977e-04,  1.3882022e-04],
        [-2.8311595e-04, -5.1688799e-05, -2.2855631e-04, ...,
         -3.5720121e-04,  1.5296042e-04,  2.5822458e-04],
        [-3.0378700e-04, -3.4179023e-04, -4.6908020e-04, ...,
         -2.9467858e-04,  1.2266744e-04,  5.0592062e-04],
        ...,
        [-7.3529821e-04,  5.5791042e-04, -1.3486024e-03, ...,
          7.0085510e-04, -4.2028888e-04,  4.9458613e-04],
        [-9.8707271e-04,  4.8461687e-04, -1.0429504e-03, ...,
          8.1573508e-04, -5.2488403e-04,  5.3699507e-04],
        [-1.3122705e-03,  4.3361785e-04, -7.3292898e-04, ...,
          9.8407397e-04, -6.9479493e-04,  6.4971799e-04]],

       [[-1.2915002e-04, -8.4298663e-05, -2.0246314e-04, ...,
         -4.0677792e-04,  1.0153977e-04,  1.3882022e-04],
        [-1.9998186e-04, -5.8593228e-05, -3.5748511e-04, ...,
         -3.6929629e-04, 

In [33]:
model.summary()

Model: "text_generator_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_7 (LSTM)                multiple                  18882560  
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
lstm_8 (LSTM)                multiple                  33562624  
_________________________________________________________________
dropout_5 (Dropout)          multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
______________________________________

In [34]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
488/488 [==============================] - 404s 822ms/step - loss: 3.8764
Epoch 2/10
488/488 [==============================] - 405s 829ms/step - loss: 2.9006
Epoch 3/10
488/488 [==============================] - 404s 828ms/step - loss: 2.5890
Epoch 4/10
488/488 [==============================] - 404s 827ms/step - loss: 2.2906
Epoch 5/10
488/488 [==============================] - 404s 828ms/step - loss: 2.0165
Epoch 6/10
488/488 [==============================] - 405s 829ms/step - loss: 1.7816
Epoch 7/10
488/488 [==============================] - 404s 828ms/step - loss: 1.5962
Epoch 8/10
488/488 [==============================] - 404s 828ms/step - loss: 1.4507
Epoch 9/10
488/488 [==============================] - 404s 827ms/step - loss: 1.3314
Epoch 10/10
488/488 [==============================] - 404s 827ms/step - loss: 1.2424


In [35]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [36]:
val_loss = model.evaluate(enc_val, dec_val, verbose=2)

977/977 - 71s - loss: 2.1455


In [37]:
generate_text(model, tokenizer, init_sentence="<start> I love")

'<start> i love you , liberian girl , <end> '

# 고찰
- hidden size를 1024에서 2048로 두배 올렸더니 loss가 줄어들었다.
  - 그렇다면 단순히 LSTM층을 하나 더 올린다거나 hidden size를 높여주는 것만으로 성능이 올라간다고 볼 수 있나?
- CV에서는 모델의 성능을 평가할 지표들이 많이있다
  - IOU나 Accuracy,품질 평가 metric SSIM, PSNR같이 여러개가 있고 직관적으로는 사람의 눈으로 확인 할 수 있다.  
  - 다양한 자연어 처리의 평가 기준을 알고싶다.
  